## 딥러닝을 이용한 자연어 처리

### 1. 텍스트의 토큰화

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
from numpy import array
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 케라스의 텍스트 전처리와 관련한 함수 중 text_to_word_sequence 함수를 불러옴
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 전처리할 텍스트를 정한다.
text = '해보지 않으면 해낼 수 없다'

# 해당 텍스트를 토큰화
result = text_to_word_sequence(text)
print("\n원문:\n",text)
print("\n토큰화:\n",result)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


* 단어 빈도수 세기

In [41]:
# 전처리하려는 세 개의 문장을 정한다.

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

# 토큰화 함수를 이요해 전처리
token = Tokenizer()     # 토큰화 함수 지정
token.fit_on_texts(docs) # 토큰화 함수에 문장 적용

# 단어의 빈도수를 계산한 결과를 각 옵션에 맞추어 출력
# Tokenizer()의 word_counts 함수는 순서를 기억하는 OrderedDict 클래스를 사용
print("\n단어 카운트:", token.word_counts)

# 출력되는 순서는 랜덤
print("\n문장 카운트:", token.document_count)
print("\n각 단어가 몇 개의 문장에 포함되어 있는가::", token.word_docs)
print("\n각 단어에 매겨진 인덱스 값:", token.word_index)




단어 카운트: OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트: 3

각 단어가 몇 개의 문장에 포함되어 있는가:: defaultdict(<class 'int'>, {'토큰화합니다': 1, '텍스트의': 2, '각': 1, '먼저': 1, '단어를': 1, '나누어': 1, '딥러닝에서': 2, '단어로': 1, '토큰화해야': 1, '인식됩니다': 1, '결과는': 1, '수': 1, '토큰화한': 1, '있습니다': 1, '사용할': 1})

각 단어에 매겨진 인덱스 값: {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


### 2. 단어의 원-핫 인코딩

In [42]:
text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text]) # 리스트 형식으로 넣는다.
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [43]:
x = token.texts_to_sequences([text]) # 텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
print(x)

[[1, 2, 3, 4, 5, 6]]


In [44]:
# 인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) +1 # +1 해주어야 함.
x = to_categorical(x, num_classes=word_size)
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


### 3. 텍스트를 읽고 긍정, 부정 예측하기

In [45]:
# 텍스트 리뷰 자료를 지정
docs = ['너무 재밌네요', '최고에요', '참 잘 만든 영화에요', '추천하고 싶은 영화입니다',
        '한번 더 보고싶네요', '글쎄요', '별로에요', '생각보다 지루하네요', '연기가 어색해', '재미없어요']

# 긍정리뷰는 1, 부정 리뷰는 0으로 클래스 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로에요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해': 19, '재미없어요': 20}


In [46]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:", x)


리뷰 텍스트, 토큰화 결과: [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [47]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 준다
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_x = pad_sequences(x, maxlen=4)
print("\n패딩 결과:\n", padded_x)



패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [50]:
# 임베딩에 입력될 단어의 수를 지정
word_size = len(token.word_index) + 1
word_size

21

Embedding Layer  
  * 단어를 고정된 길이의 밀집 벡터로 매핑하는 역할을 하며, 이를 통해 단어 간의 의미적 유사성을 반영하면서 모델이 텍스트 데이터를 더 잘 이해하고 처리할 수 있도록 돕는다.
  * Embedding() 함수의 인자  
    * word_size: 이 값은 단어 집합의 크기를 의미
    * 8 : 임베딩 레이어에서 각 단어를 표현하는 벡터의 차원 크기를 의미.하이퍼파라미터. 차원이 작으면 계산 비용이 적지만, 단어의 의미를 잘 표현하지 못할 수 있고, 차원이 크면 계산 비용이 커지지만 단어의 의미를 더 잘 표현
    * input_length : 입력 시퀀스의 길이. 각 문장의 최대 단어 개수. pad_sequences() 함수에서 지정한 maxlen 길이와 같다.

In [51]:
# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 4, 8)              168       
                                                                 
 flatten_3 (Flatten)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 911ms/step - loss: 0.6922 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6904 - accuracy: 0.7000
Epoch 3/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6886 - accuracy: 0.7000
Epoch 4/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6868 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6849 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6831 - accuracy: 0.9000
Epoch 7/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6813 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6795 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6777 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6759 - accuracy: 0.9000
Epoch 11/20
1/1 [===========